# Supervised Learning - part 2

- Only the best model, all data.

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

from sklearn import naive_bayes, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, make_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from imblearn.over_sampling import RandomOverSampler

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
RANDOM_STATE = 42

In [4]:
lemmas = pd.read_csv('../data/lemmas.csv')
lemmas

,book_poem_id,lemmas
0,1_0001-0000-0000-0001-0000,"[('tvůj', 'P'), ('loď', 'N'), ('jít', 'V'), ('..."
1,1_0001-0001-0000-0001-0000,"[('už', 'D'), ('míza', 'N'), ('naplňovat', 'V'..."
2,1_0001-0001-0000-0002-0000,"[('už', 'D'), ('jaro', 'N'), ('výskat', 'V'), ..."
3,1_0001-0001-0000-0003-0000,"[('teď', 'D'), ('slunce', 'N'), ('projít', 'V'..."
4,1_0001-0001-0000-0004-0000,"[('skála', 'N'), ('nebetyčný', 'A'), ('shluk',..."
...,...,...
66423,1689_0001-0000-0000-0026-0000,"[('u', 'R'), ('altán', 'N'), ('bílý', 'A'), ('..."
66424,1689_0001-0000-0000-0027-0000,"[('nad', 'R'), ('tůně', 'N'), ('honit', 'V'), ..."
66425,1689_0001-0000-0000-0028-0000,"[('podle', 'R'), ('cesta', 'N'), ('hloh', 'N')..."
66426,1689_0001-0000-0000-0029-0000,"[('na', 'R'), ('lesní', 'A'), ('stráň', 'N'), ..."


In [5]:
annotations = pd.read_csv('../data/poem_value_save.csv')
annotations

,book_poem_id,poem_id,book_id,id_topic,value,name
0,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,1,-1,Dětství/mateřství/rodičovství
1,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,2,-1,Domov
2,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,3,-1,Exotika/cestovatelství
3,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,4,-1,Chudoba/útisk
4,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,9,-1,Náboženství/víra
...,...,...,...,...,...,...
27154,1675_0001-0009-0000-0015-0002,0001-0009-0000-0015-0002,1675,26,-1,Město
27155,1675_0001-0009-0000-0015-0002,0001-0009-0000-0015-0002,1675,27,-1,Venkov
27156,1675_0001-0009-0000-0015-0002,0001-0009-0000-0015-0002,1675,28,-1,Národ/vlast
27157,1675_0001-0009-0000-0015-0002,0001-0009-0000-0015-0002,1675,29,-1,Historie


In [6]:
annotations_1 = annotations[annotations.value == 1].reset_index(drop=True)
annotations_1

,book_poem_id,poem_id,book_id,id_topic,value,name
0,2_0001-0000-0000-0005-0000,0001-0000-0000-0005-0000,2,13,1,Práce
1,2_0001-0000-0000-0008-0000,0001-0000-0000-0008-0000,2,9,1,Náboženství/víra
2,3_0001-0003-0000-0012-0000,0001-0003-0000-0012-0000,3,9,1,Náboženství/víra
3,7_0001-0000-0000-0065-0000,0001-0000-0000-0065-0000,7,20,1,"Umění/literatura, poezie/tvorba"
4,8_0001-0000-0000-0004-0000,0001-0000-0000-0004-0000,8,30,1,Sebereflexe/vnitřní život
...,...,...,...,...,...,...
917,1675_0001-0005-0000-0028-0000,0001-0005-0000-0028-0000,1675,14,1,Příroda
918,1675_0001-0005-0000-0042-0000,0001-0005-0000-0042-0000,1675,30,1,Sebereflexe/vnitřní život
919,1675_0001-0007-0001-0003-0000,0001-0007-0001-0003-0000,1675,12,1,Politika/společnost
920,1675_0001-0007-0001-0003-0000,0001-0007-0001-0003-0000,1675,26,1,Město


In [7]:
annotations_lemmas = pd.merge(lemmas, annotations_1, on='book_poem_id', how='left')
annotations_lemmas

,book_poem_id,lemmas,poem_id,book_id,id_topic,value,name
0,1_0001-0000-0000-0001-0000,"[('tvůj', 'P'), ('loď', 'N'), ('jít', 'V'), ('...",NaN,NaN,NaN,NaN,NaN
1,1_0001-0001-0000-0001-0000,"[('už', 'D'), ('míza', 'N'), ('naplňovat', 'V'...",NaN,NaN,NaN,NaN,NaN
2,1_0001-0001-0000-0002-0000,"[('už', 'D'), ('jaro', 'N'), ('výskat', 'V'), ...",NaN,NaN,NaN,NaN,NaN
3,1_0001-0001-0000-0003-0000,"[('teď', 'D'), ('slunce', 'N'), ('projít', 'V'...",NaN,NaN,NaN,NaN,NaN
4,1_0001-0001-0000-0004-0000,"[('skála', 'N'), ('nebetyčný', 'A'), ('shluk',...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
66501,1689_0001-0000-0000-0026-0000,"[('u', 'R'), ('altán', 'N'), ('bílý', 'A'), ('...",NaN,NaN,NaN,NaN,NaN
66502,1689_0001-0000-0000-0027-0000,"[('nad', 'R'), ('tůně', 'N'), ('honit', 'V'), ...",NaN,NaN,NaN,NaN,NaN
66503,1689_0001-0000-0000-0028-0000,"[('podle', 'R'), ('cesta', 'N'), ('hloh', 'N')...",NaN,NaN,NaN,NaN,NaN
66504,1689_0001-0000-0000-0029-0000,"[('na', 'R'), ('lesní', 'A'), ('stráň', 'N'), ...",NaN,NaN,NaN,NaN,NaN


# Filter the data

- Filter out all POS but Nouns (N), Adjectives (A) and Verbs (V)

- Filter out frequent (stop)words that does not carry any extra semantic information

In [8]:
stopwords = ['mít', 'jít', 'být', 'dát', 'moci', 'chtít']

In [9]:
%%time

lemmas_filtered_column = []

# extract lemmas with N, A or V POS tag
for index, poem in annotations_lemmas.iterrows():
    if index % 1000 == 0:
        print(index, end=' ')
    
    lemmas_filtered_poem = []
    lemmas_poem = literal_eval(poem['lemmas'])
    for lemma_pos in lemmas_poem:
        lemma, pos = lemma_pos
        if pos in ['N', 'A', 'V'] and lemma not in stopwords:
            lemmas_filtered_poem.append(lemma)
    lemmas_filtered_column.append(lemmas_filtered_poem)

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 CPU times: user 1min 52s, sys: 1.28 s, total: 1min 53s
Wall time: 1min 54s


In [10]:
annotations_lemmas['lemmas_filtered'] = lemmas_filtered_column
annotations_lemmas

,book_poem_id,lemmas,poem_id,book_id,id_topic,value,name,lemmas_filtered
0,1_0001-0000-0000-0001-0000,"[('tvůj', 'P'), ('loď', 'N'), ('jít', 'V'), ('...",NaN,NaN,NaN,NaN,NaN,"[loď, vysoký, moře, brázda, stříbro, rej, přij..."
1,1_0001-0001-0000-0001-0000,"[('už', 'D'), ('míza', 'N'), ('naplňovat', 'V'...",NaN,NaN,NaN,NaN,NaN,"[míza, naplňovat, stonek, přijít, modrý, anemo..."
2,1_0001-0001-0000-0002-0000,"[('už', 'D'), ('jaro', 'N'), ('výskat', 'V'), ...",NaN,NaN,NaN,NaN,NaN,"[jaro, výskat, potkání, kos, pět, zelený, strá..."
3,1_0001-0001-0000-0003-0000,"[('teď', 'D'), ('slunce', 'N'), ('projít', 'V'...",NaN,NaN,NaN,NaN,NaN,"[slunce, projít, poledník, znamení, mžik, stro..."
4,1_0001-0001-0000-0004-0000,"[('skála', 'N'), ('nebetyčný', 'A'), ('shluk',...",NaN,NaN,NaN,NaN,NaN,"[skála, nebetyčný, shluk, řada, vystupovat, pá..."
...,...,...,...,...,...,...,...,...
66501,1689_0001-0000-0000-0026-0000,"[('u', 'R'), ('altán', 'N'), ('bílý', 'A'), ('...",NaN,NaN,NaN,NaN,NaN,"[altán, bílý, jasmín, opojivý, vůně, dýchat, z..."
66502,1689_0001-0000-0000-0027-0000,"[('nad', 'R'), ('tůně', 'N'), ('honit', 'V'), ...",NaN,NaN,NaN,NaN,NaN,"[tůně, honit, černý, mrak, rok, voda, klonit, ..."
66503,1689_0001-0000-0000-0028-0000,"[('podle', 'R'), ('cesta', 'N'), ('hloh', 'N')...",NaN,NaN,NaN,NaN,NaN,"[cesta, hloh, keř, sladkohořký, vůně, duch, ňa..."
66504,1689_0001-0000-0000-0029-0000,"[('na', 'R'), ('lesní', 'A'), ('stráň', 'N'), ...",NaN,NaN,NaN,NaN,NaN,"[lesní, stráň, zalitý, slunce, rosa, třpytit, ..."


In [11]:
def c_tf_idf(documents, m, ngram_range=(1, 1)):    
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)
    
    return tf_idf, count


def extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, groupby, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_cluster[groupby])
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words


def extract_topic_sizes(df, groupby):
    topic_sizes = (df.groupby([groupby])
                     .preprocessed_text
                     .count()
                     .reset_index()
                     .rename({"preprocessed_text": "size"}, axis='columns')
                     .sort_values("size", ascending=False))
    return topic_sizes

In [12]:
annotations_lemmas = annotations_lemmas.drop(['name'], axis=1)

In [13]:
annotations_lemmas.head(1)

,book_poem_id,lemmas,poem_id,book_id,id_topic,value,lemmas_filtered
0,1_0001-0000-0000-0001-0000,"[('tvůj', 'P'), ('loď', 'N'), ('jít', 'V'), ('...",NaN,NaN,NaN,NaN,"[loď, vysoký, moře, brázda, stříbro, rej, přij..."


# TFIDF classification

In [14]:
data = pd.DataFrame()
data = annotations_lemmas[['book_poem_id', 'id_topic', 'lemmas_filtered']]
data['preprocessed_text'] = annotations_lemmas['lemmas_filtered'].apply(lambda x: ' '.join(x))
data

,book_poem_id,id_topic,lemmas_filtered,preprocessed_text
0,1_0001-0000-0000-0001-0000,NaN,"[loď, vysoký, moře, brázda, stříbro, rej, přij...",loď vysoký moře brázda stříbro rej přijít modr...
1,1_0001-0001-0000-0001-0000,NaN,"[míza, naplňovat, stonek, přijít, modrý, anemo...",míza naplňovat stonek přijít modrý anemonka me...
2,1_0001-0001-0000-0002-0000,NaN,"[jaro, výskat, potkání, kos, pět, zelený, strá...",jaro výskat potkání kos pět zelený stráň mlíčí...
3,1_0001-0001-0000-0003-0000,NaN,"[slunce, projít, poledník, znamení, mžik, stro...",slunce projít poledník znamení mžik stroj hvěz...
4,1_0001-0001-0000-0004-0000,NaN,"[skála, nebetyčný, shluk, řada, vystupovat, pá...",skála nebetyčný shluk řada vystupovat pásmo le...
...,...,...,...,...
66501,1689_0001-0000-0000-0026-0000,NaN,"[altán, bílý, jasmín, opojivý, vůně, dýchat, z...",altán bílý jasmín opojivý vůně dýchat zkolébáv...
66502,1689_0001-0000-0000-0027-0000,NaN,"[tůně, honit, černý, mrak, rok, voda, klonit, ...",tůně honit černý mrak rok voda klonit leknínov...
66503,1689_0001-0000-0000-0028-0000,NaN,"[cesta, hloh, keř, sladkohořký, vůně, duch, ňa...",cesta hloh keř sladkohořký vůně duch ňadro div...
66504,1689_0001-0000-0000-0029-0000,NaN,"[lesní, stráň, zalitý, slunce, rosa, třpytit, ...",lesní stráň zalitý slunce rosa třpytit růžový ...


In [15]:
X_train2 = data[data['id_topic'].notnull()]
X_rest = data[~data['id_topic'].notnull()]

In [16]:
X = X_train2[['preprocessed_text']].copy()
y = X_train2[['id_topic']].copy()

In [17]:
X_rest2 = X_rest[['preprocessed_text']].copy()
y_rest2 = X_rest[['id_topic']].copy()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [19]:
print(len(X_train), len(X_test))

645 277


In [20]:
y.value_counts()

id_topic
25.0        171
30.0        111
9.0          85
14.0         85
17.0         77
12.0         64
20.0         64
28.0         51
16.0         37
29.0         32
1.0          29
13.0         16
4.0          16
2.0          15
26.0         13
24.0          9
19.0          9
15.0          8
21.0          8
23.0          8
22.0          4
27.0          4
3.0           4
18.0          2
dtype: int64

# Scores of annotated data

In [21]:
X_train2

,book_poem_id,id_topic,lemmas_filtered,preprocessed_text
47,2_0001-0000-0000-0005-0000,13.0,"[chuť, práce, milý, dítě, bůh, práce, síla, ďá...",chuť práce milý dítě bůh práce síla ďáblův pad...
50,2_0001-0000-0000-0008-0000,9.0,"[truchlit, rána, bolný, lidský, zloba, usoudit...",truchlit rána bolný lidský zloba usoudit ruka ...
139,3_0001-0003-0000-0012-0000,9.0,"[srdce, Páně, velechrám, svatý, skvít, záře, k...",srdce Páně velechrám svatý skvít záře kříž zna...
279,7_0001-0000-0000-0065-0000,20.0,"[lid, chránit, stánek, chudý, slavíček, smutný...",lid chránit stánek chudý slavíček smutný hlas ...
303,8_0001-0000-0000-0004-0000,30.0,"[plout, loď, daleký, vodní, pláň, modrý, dálka...",plout loď daleký vodní pláň modrý dálka štěstí...
...,...,...,...,...
66297,1675_0001-0005-0000-0028-0000,14.0,"[duše, rozjásaný, slavný, zachytit, zář, korun...",duše rozjásaný slavný zachytit zář koruna habr...
66311,1675_0001-0005-0000-0042-0000,30.0,"[smutný, srdce, zatoulat, bledý, soumrak, vůně...",smutný srdce zatoulat bledý soumrak vůně jeseň...
66337,1675_0001-0007-0001-0003-0000,12.0,"[nalíčený, žena, zrak, hořet, tmít, obočí, šíj...",nalíčený žena zrak hořet tmít obočí šíj rána n...
66338,1675_0001-0007-0001-0003-0000,26.0,"[nalíčený, žena, zrak, hořet, tmít, obočí, šíj...",nalíčený žena zrak hořet tmít obočí šíj rána n...


In [86]:
# Create the library
min_frequency = 20
dictionary = Dictionary(X_train2['lemmas_filtered'])
# Filter out words
dictionary.filter_extremes(no_below=min_frequency)

In [87]:
from collections import Counter

# create word frequency count
word_counts = Counter(word for row in X_train2['lemmas_filtered'] for word in row)

# filter lists based on word frequency count
X_train2['lemmas_filtered_a'] = X_train2['lemmas_filtered'].apply(lambda x: [word for word in x if word_counts[word] >= 20])

In [88]:
X_train2['preprocessed_text'] = X_train2['lemmas_filtered_a'].apply(lambda x: ' '.join(x))

In [89]:
# merge documents in cluster into one document
docs_per_cluster = X_train2.groupby(['id_topic'], as_index = False).agg({'preprocessed_text': ' '.join})

In [90]:
docs_per_cluster

,id_topic,preprocessed_text
0,1.0,děcko hoch pustý ticho noc smrt stát sklonit k...
1,2.0,sedět hučet sníh padat vzpomínat cos šeptat hl...
2,3.0,země dávný hlubina žena slza smutek vracet pís...
3,4.0,oř oř mladý hoch stráň kráčet divný myšlénka h...
4,9.0,rána lidský zloba ruka pán jmout rána hlas vol...
5,12.0,rok studený rok horký stát moudrost osud čin v...
6,13.0,práce milý dítě bůh práce síla padat práce prá...
7,14.0,klín noc mladý den celý spánek vstát spící zem...
8,15.0,černý pták rok držet tvář bít hruď zrak tělo r...
9,16.0,hvězda hořet cítit život naděje kalný voda hoř...


In [91]:
tf_idf, count = c_tf_idf(docs_per_cluster.preprocessed_text.values, m=len(X_train2))

In [92]:
top_n_words = extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, 'id_topic', n=10)

In [93]:
topic_sizes = extract_topic_sizes(X_train2, 'id_topic')
topic_sizes

,id_topic,size
18,25.0,171
23,30.0,111
4,9.0,85
7,14.0,85
10,17.0,77
5,12.0,64
13,20.0,64
21,28.0,51
9,16.0,37
22,29.0,32


In [94]:
topics = []

for i in top_n_words:
    words_topic = []
    for word in top_n_words[i]:
        words_topic.append(word[0])
    topics.append(words_topic)

In [95]:
cm = CoherenceModel(topics=topics, topn=10, dictionary=dictionary, texts=X_train2['lemmas_filtered_a'], coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4302181309505948

In [96]:
from utils import topic_diversity

topic_diversity(topics, 10)

0.8458333333333333

In [97]:
top_n_words

{1.0: [('matka', 0.04037930037061493),
  ('táta', 0.03376806058779616),
  ('dítě', 0.023939725488320247),
  ('máma', 0.02317174471126696),
  ('syn', 0.02058511187019645),
  ('děcko', 0.02021411140813173),
  ('zpívat', 0.019780078032076487),
  ('říci', 0.01769962622408931),
  ('čelo', 0.017183563492242364),
  ('boží', 0.017078879613977343)],
 2.0: [('domov', 0.02758365904337707),
  ('přítel', 0.026777374781291923),
  ('zahrada', 0.02471080919864803),
  ('okno', 0.024304112990243758),
  ('sníh', 0.02403623762358073),
  ('chata', 0.023069068797671528),
  ('vidět', 0.022603872969002098),
  ('nízký', 0.02254793374180246),
  ('střecha', 0.02254793374180246),
  ('najít', 0.019813170175064083)],
 3.0: [('hlubina', 0.0890782567577381),
  ('písek', 0.0881098440872867),
  ('celý', 0.053750622810089944),
  ('moře', 0.05076199010101059),
  ('touha', 0.04953737427040881),
  ('rovný', 0.04669163933127371),
  ('uzřít', 0.04669163933127371),
  ('ruch', 0.04611731815059602),
  ('pohár', 0.04556853095836

# Oversampling

In [33]:
# Apply random oversampling to the minority class
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=RANDOM_STATE)
X_train_sampled, y_train_sampled = ros.fit_resample(X_train, y_train)

In [34]:
y_train_sampled.value_counts()

id_topic
1.0         120
2.0         120
29.0        120
28.0        120
27.0        120
26.0        120
25.0        120
24.0        120
23.0        120
22.0        120
21.0        120
20.0        120
19.0        120
18.0        120
17.0        120
16.0        120
15.0        120
14.0        120
13.0        120
12.0        120
9.0         120
4.0         120
3.0         120
30.0        120
dtype: int64

# Encode target variable

In [35]:
X_train = X_train_sampled
y_train = y_train_sampled

In [36]:
# Encoding of target variable - convert text labels into numbers
encoder = LabelEncoder()
encoder_fitted = encoder.fit(y_train)

In [37]:
y_train = encoder_fitted.transform(y_train)
y_test = encoder_fitted.transform(y_test)

In [38]:
encoder_fitted.classes_

array([ 1.,  2.,  3.,  4.,  9., 12., 13., 14., 15., 16., 17., 18., 19.,
       20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.])

In [39]:
encoder_fitted.transform([ 1,  2,  3,  4,  9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

# Vectorization

In [40]:
vectorizer_tfidf = TfidfVectorizer()  # good for SVM
#vectorizer_count = CountVectorizer()  # good for Naive Bayes

vectorizer_fitted_tfidf = vectorizer_tfidf.fit(X_rest2['preprocessed_text'])
#vectorizer_fitted_count = vectorizer_count.fit(X_train['preprocessed_text'])

In [41]:
train_X_encodings_tfidf = vectorizer_fitted_tfidf.transform(X_train['preprocessed_text'])
test_X_encodings_tfidf = vectorizer_fitted_tfidf.transform(X_test['preprocessed_text'])
rest_X_encodings_tfidf = vectorizer_fitted_tfidf.transform(X_rest2['preprocessed_text'])

#train_X_encodings_count = vectorizer_fitted_count.transform(X_train['preprocessed_text'])
#test_X_encodings_count = vectorizer_fitted_count.transform(X_test['preprocessed_text'])

In [42]:
train_X_encodings_tfidf

<2880x64161 sparse matrix of type '<class 'numpy.float64'>'
	with 190042 stored elements in Compressed Sparse Row format>

In [43]:
#train_X_embeddings_word = X_train['embedding_word'].to_list()
#test_X_embeddings_word = X_test['embedding_word'].to_list()

# ML

In [44]:
f1_macro_scorer = make_scorer(f1_score, average='macro')

In [45]:
scoring = 'accuracy'

In [46]:
# Create k-fold cross-validation object
kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# The best model

In [47]:
# MANUALLY pick the best performing model
model_svm = MLPClassifier(activation= 'logistic', alpha=0.01, random_state=RANDOM_STATE) #probability=True
# train your model on the entire training set
model_svm.fit(train_X_encodings_tfidf, y_train)

MLPClassifier(activation='logistic', alpha=0.01, random_state=42)

In [48]:
# predict labels for the test set
y_pred = model_svm.predict(test_X_encodings_tfidf)

In [49]:
#print(classification_report(y_test, y_pred, labels=encoder_fitted.classes_))

In [50]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.30685920577617326

In [51]:
f1_score(y_true=y_test, y_pred=y_pred, average='macro')

0.12412276030522258

In [52]:
# on all set
y_pred_rest = model_svm.predict(rest_X_encodings_tfidf)

In [53]:
y_pred_rest

array([18, 18,  7, ..., 18,  7,  7])

In [54]:
X_rest['predicted_label'] = y_pred_rest

In [55]:
X_rest

,book_poem_id,id_topic,lemmas_filtered,preprocessed_text,predicted_label
0,1_0001-0000-0000-0001-0000,NaN,"[loď, vysoký, moře, brázda, stříbro, rej, přij...",loď vysoký moře brázda stříbro rej přijít modr...,18
1,1_0001-0001-0000-0001-0000,NaN,"[míza, naplňovat, stonek, přijít, modrý, anemo...",míza naplňovat stonek přijít modrý anemonka me...,18
2,1_0001-0001-0000-0002-0000,NaN,"[jaro, výskat, potkání, kos, pět, zelený, strá...",jaro výskat potkání kos pět zelený stráň mlíčí...,7
3,1_0001-0001-0000-0003-0000,NaN,"[slunce, projít, poledník, znamení, mžik, stro...",slunce projít poledník znamení mžik stroj hvěz...,23
4,1_0001-0001-0000-0004-0000,NaN,"[skála, nebetyčný, shluk, řada, vystupovat, pá...",skála nebetyčný shluk řada vystupovat pásmo le...,7
...,...,...,...,...,...
66501,1689_0001-0000-0000-0026-0000,NaN,"[altán, bílý, jasmín, opojivý, vůně, dýchat, z...",altán bílý jasmín opojivý vůně dýchat zkolébáv...,18
66502,1689_0001-0000-0000-0027-0000,NaN,"[tůně, honit, černý, mrak, rok, voda, klonit, ...",tůně honit černý mrak rok voda klonit leknínov...,23
66503,1689_0001-0000-0000-0028-0000,NaN,"[cesta, hloh, keř, sladkohořký, vůně, duch, ňa...",cesta hloh keř sladkohořký vůně duch ňadro div...,18
66504,1689_0001-0000-0000-0029-0000,NaN,"[lesní, stráň, zalitý, slunce, rosa, třpytit, ...",lesní stráň zalitý slunce rosa třpytit růžový ...,7


In [ ]:
# Create the dictionary
min_frequency = 10
dictionary = Dictionary(X_rest['lemmas_filtered'])
# Filter out words
dictionary.filter_extremes(no_below=min_frequency)

In [74]:
from collections import Counter

# create word frequency count
word_counts = Counter(word for row in X_rest['lemmas_filtered'] for word in row)

# filter lists based on word frequency count
X_rest['lemmas_filtered_a'] = X_rest['lemmas_filtered'].apply(lambda x: [word for word in x if word_counts[word] >= 20])

In [75]:
X_rest['lemmas_filtered_a'] = X_rest['lemmas_filtered_a'].apply(lambda x: ' '.join(x))

In [76]:
mapping = dict(zip(encoder_fitted.transform(encoder_fitted.classes_), encoder_fitted.classes_))

In [77]:
X_rest.predicted_label.replace(mapping, inplace=True)

In [78]:
X_rest

,book_poem_id,id_topic,lemmas_filtered,preprocessed_text,predicted_label,lemmas_filtered_a
0,1_0001-0000-0000-0001-0000,NaN,"[loď, vysoký, moře, brázda, stříbro, rej, přij...",loď vysoký moře brázda stříbro rej přijít modr...,25,loď vysoký moře brázda stříbro rej přijít modr...
1,1_0001-0001-0000-0001-0000,NaN,"[míza, naplňovat, stonek, přijít, modrý, anemo...",míza naplňovat stonek přijít modrý anemonka me...,25,míza naplňovat stonek přijít modrý anemonka me...
2,1_0001-0001-0000-0002-0000,NaN,"[jaro, výskat, potkání, kos, pět, zelený, strá...",jaro výskat potkání kos pět zelený stráň mlíčí...,21,jaro výskat potkání kos pět zelený stráň pozla...
3,1_0001-0001-0000-0003-0000,NaN,"[slunce, projít, poledník, znamení, mžik, stro...",slunce projít poledník znamení mžik stroj hvěz...,30,slunce projít znamení mžik stroj hvězdárna hlá...
4,1_0001-0001-0000-0004-0000,NaN,"[skála, nebetyčný, shluk, řada, vystupovat, pá...",skála nebetyčný shluk řada vystupovat pásmo le...,21,skála nebetyčný shluk řada vystupovat pásmo le...
...,...,...,...,...,...,...
66501,1689_0001-0000-0000-0026-0000,NaN,"[altán, bílý, jasmín, opojivý, vůně, dýchat, z...",altán bílý jasmín opojivý vůně dýchat zkolébáv...,25,altán bílý jasmín opojivý vůně dýchat sladký k...
66502,1689_0001-0000-0000-0027-0000,NaN,"[tůně, honit, černý, mrak, rok, voda, klonit, ...",tůně honit černý mrak rok voda klonit leknínov...,30,tůně honit černý mrak rok voda klonit leknínov...
66503,1689_0001-0000-0000-0028-0000,NaN,"[cesta, hloh, keř, sladkohořký, vůně, duch, ňa...",cesta hloh keř sladkohořký vůně duch ňadro div...,25,cesta hloh keř vůně duch ňadro divý vášeň srdc...
66504,1689_0001-0000-0000-0029-0000,NaN,"[lesní, stráň, zalitý, slunce, rosa, třpytit, ...",lesní stráň zalitý slunce rosa třpytit růžový ...,21,lesní stráň zalitý slunce rosa třpytit růžový ...


In [79]:
# merge documents in cluster into one document
docs_per_cluster = X_rest.groupby(['predicted_label'], as_index = False).agg({'preprocessed_text': ' '.join})
docs_per_cluster

,predicted_label,preprocessed_text
0,2,dudy starý píseň nota známý chasa noha táta pa...
1,3,domov domov chorál veleba hřímat ňadro zvedat ...
2,9,berle strnulý noha vléci slabý noha děcko oko ...
3,16,klekání odzvonit spát říci bába sedět chalupa ...
4,19,bernardýn čistý krev přijít svět park široširý...
5,20,hoch práce otec dílo vést dílna začazený dřít ...
6,21,jaro výskat potkání kos pět zelený stráň mlíčí...
7,22,rachotit hrom skrýše oblak pár kapat vroucí mr...
8,23,Rek poutník zamlklý tajemství věčný hroužit zr...
9,24,síň vrazit celý uřícený pláč hořký běžet kuchy...


In [80]:
tf_idf, count = c_tf_idf(docs_per_cluster.preprocessed_text.values, m=len(X_rest))

In [81]:
top_n_words = extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, 'predicted_label', n=10)

In [82]:
topic_sizes = extract_topic_sizes(X_rest, 'predicted_label')
topic_sizes

,predicted_label,size
10,25,29273
15,30,10688
3,16,7443
6,21,4573
4,19,4111
12,27,3240
9,24,2958
13,28,2351
8,23,323
14,29,274


In [83]:
top_n_words

{2: [('dítě', 0.04162754261256442),
  ('táta', 0.037025541681604934),
  ('hoch', 0.02176017384990982),
  ('matka', 0.019237393003240306),
  ('matička', 0.01247697355142294),
  ('otec', 0.010802720675758317),
  ('letět', 0.010408886345447127),
  ('dcerka', 0.01029386744460883),
  ('hrát', 0.0099427447326541),
  ('děcko', 0.009690663605657391)],
 3: [('domov', 0.35908261360580435),
  ('vlast', 0.04683428125585237),
  ('český', 0.036997329387882194),
  ('drahý', 0.030572598696591582),
  ('polétávat', 0.025207400002977016),
  ('poutník', 0.021504931234452376),
  ('čin', 0.020727117883900303),
  ('křen', 0.020568999788701404),
  ('krásný', 0.02004834082879479),
  ('ráj', 0.01826726549405279)],
 9: [('bída', 0.061008704030531595),
  ('panský', 0.03275299656668598),
  ('vácslav', 0.022898429936253878),
  ('občan', 0.0223410554310705),
  ('vrata', 0.019752172687004455),
  ('pole', 0.017564073054422265),
  ('mlýn', 0.01690913460516466),
  ('dvůr', 0.016061407354478644),
  ('děd', 0.015885299083

In [72]:
df_topics = pd.read_csv('../data/list_of_topics.csv')
df_topics

,id_topic,name
0,1,Dětství/mateřství/rodičovství
1,2,Domov
2,3,Exotika/cestovatelství
3,4,Chudoba/útisk
4,9,Náboženství/víra
5,11,Pokrok/technika
6,12,Politika/společnost
7,13,Práce
8,14,Příroda
9,15,Revoluce/Osvobození


In [68]:
#for i in [k for k in top_n_words]:
#    print(i, df_topics[df_topics.id_topic == i].reset_index(drop=True).name[0])
#    word_cloud(top_n_words, i)

In [84]:
topics = []

for i in top_n_words:
    words_topic = []
    for word in top_n_words[i]:
        words_topic.append(word[0])
    topics.append(words_topic)
    
topics[2]

['bída',
 'panský',
 'vácslav',
 'občan',
 'vrata',
 'pole',
 'mlýn',
 'dvůr',
 'děd',
 'moře']

In [70]:
cm = CoherenceModel(topics=topics, topn=10, dictionary=dictionary, texts=X_rest['lemmas_filtered'], coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4503500177057682

In [71]:
topic_diversity(topics, 10)

0.8714285714285714